Gerekli Importları ekliyoruz.

In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score,RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.linear_model import RidgeClassifier,LogisticRegression
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')




Dataset ekliyoruz. Bu verisetinde yapmaya çalıştığım şey verilen diğer bilgilere göre kullanıcının rezarvazyonu tamamlayacakmı  yoksa tamamlamadan çıkacakmı diye tahmin etmeye çalışacağım.

In [16]:
df=pd.read_csv('data/customer_booking.csv',encoding='latin-1')
df.head(10)

,num_passengers,sales_channel,trip_type,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete
0,2,Internet,RoundTrip,262,19,7,Sat,AKLDEL,New Zealand,1,0,0,5.52,0
1,1,Internet,RoundTrip,112,20,3,Sat,AKLDEL,New Zealand,0,0,0,5.52,0
2,2,Internet,RoundTrip,243,22,17,Wed,AKLDEL,India,1,1,0,5.52,0
3,1,Internet,RoundTrip,96,31,4,Sat,AKLDEL,New Zealand,0,0,1,5.52,0
4,2,Internet,RoundTrip,68,22,15,Wed,AKLDEL,India,1,0,1,5.52,0
5,1,Internet,RoundTrip,3,48,20,Thu,AKLDEL,New Zealand,1,0,1,5.52,0
6,3,Internet,RoundTrip,201,33,6,Thu,AKLDEL,New Zealand,1,0,1,5.52,0
7,2,Internet,RoundTrip,238,19,14,Mon,AKLDEL,India,1,0,1,5.52,0
8,1,Internet,RoundTrip,80,22,4,Mon,AKLDEL,New Zealand,0,0,1,5.52,0
9,1,Mobile,RoundTrip,378,30,12,Sun,AKLDEL,India,0,0,0,5.52,0


Görüldüğü gibi hiçbir null veri bulunlunmuyor o yüzden bir ekelme yada çıakrtma işlmei yapmamıza gerek kalmıyor 

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   num_passengers         50000 non-null  int64  
 1   sales_channel          50000 non-null  object 
 2   trip_type              50000 non-null  object 
 3   purchase_lead          50000 non-null  int64  
 4   length_of_stay         50000 non-null  int64  
 5   flight_hour            50000 non-null  int64  
 6   flight_day             50000 non-null  object 
 7   route                  50000 non-null  object 
 8   booking_origin         50000 non-null  object 
 9   wants_extra_baggage    50000 non-null  int64  
 10  wants_preferred_seat   50000 non-null  int64  
 11  wants_in_flight_meals  50000 non-null  int64  
 12  flight_duration        50000 non-null  float64
 13  booking_complete       50000 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 5.3+ 

Rota ve Rezervasyon ülkesi sütünların çok fazla benzersiz veri var ve bu ML için kafa karıştırıcı olabilir bu nedenle bunların en büyük 15 tanesi alıyoeuz ve diğerleini other olarak dolduruyoruz.

In [21]:
len(df['route'].unique())

799

In [22]:
country_list = df['booking_origin'].value_counts().nlargest(15).index
df['booking_origin'] = df['booking_origin'].apply(lambda x: x if x in country_list else 'Other')
route_list = df['route'].value_counts().nlargest(10)
df['route'] = df['route'].apply(lambda x: x if x in route_list else 'Other')

şimdi verileri dummie etme zamanı bu yaptığımız ile ML için uygun hale getireceğiz veri setimizi. İleride bir tahmin edilmek istenirse daha kolay erişebilmek adına burada OneHotEncoder kullanacağım.

df['route'].unique().__len__()


OUTPUT : 799 Görüldüğü gibi çok fazla benzersiz değer var bunlara tahmin etmek istediğim zaman daha kolay ulaşabilemek adına OneHotEncoding Kullanacağım.

In [23]:
ohe=OneHotEncoder()
ohed=ohe.fit_transform(df[['sales_channel','trip_type','flight_day','route','booking_origin']]).toarray()

In [24]:
ohe.get_feature_names_out()

array(['sales_channel_Internet', 'sales_channel_Mobile',
       'trip_type_CircleTrip', 'trip_type_OneWay', 'trip_type_RoundTrip',
       'flight_day_Fri', 'flight_day_Mon', 'flight_day_Sat',
       'flight_day_Sun', 'flight_day_Thu', 'flight_day_Tue',
       'flight_day_Wed', 'route_AKLKUL', 'route_DMKKIX', 'route_DMKOOL',
       'route_DMKPER', 'route_DPSICN', 'route_ICNSIN', 'route_ICNSYD',
       'route_MELPEN', 'route_MELSGN', 'route_Other', 'route_PENTPE',
       'booking_origin_Australia', 'booking_origin_China',
       'booking_origin_Hong Kong', 'booking_origin_India',
       'booking_origin_Indonesia', 'booking_origin_Japan',
       'booking_origin_Macau', 'booking_origin_Malaysia',
       'booking_origin_New Zealand', 'booking_origin_Other',
       'booking_origin_Singapore', 'booking_origin_South Korea',
       'booking_origin_Taiwan', 'booking_origin_Thailand',
       'booking_origin_United States', 'booking_origin_Vietnam'],
      dtype=object)

Şimdi bunları veri setine eklemek için bir dataframe ekleyeceğim

In [25]:
ohed = pd.DataFrame(ohed,columns=ohe.get_feature_names_out())
ohed.head()

,sales_channel_Internet,sales_channel_Mobile,trip_type_CircleTrip,trip_type_OneWay,trip_type_RoundTrip,flight_day_Fri,flight_day_Mon,flight_day_Sat,flight_day_Sun,flight_day_Thu,...,booking_origin_Macau,booking_origin_Malaysia,booking_origin_New Zealand,booking_origin_Other,booking_origin_Singapore,booking_origin_South Korea,booking_origin_Taiwan,booking_origin_Thailand,booking_origin_United States,booking_origin_Vietnam
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Şimdi bunu bizim verisetimize ekelyeceğiz.

In [26]:
df.drop(columns=['sales_channel','trip_type','flight_day','route','booking_origin'], inplace=True)

In [27]:
df[ohed.columns]=ohed

Şu anda ML için uygun hale geldi  Artık test ve train oluşturabiliriz. 50000 veri olduğu için %80 yeterli diye düşünüyorum.


In [51]:
y=df['booking_complete']
x=df.drop('booking_complete', axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.80, random_state=43)

Artık model oluşturabiliriz öncelikle RandomForestClassifier kullanacağım.

In [52]:
rf=RandomForestClassifier()
model = rf.fit(x_train, y_train)
model.score(x_test, y_test)

0.8475

In [53]:
rf=RandomForestClassifier(n_estimators=200)
model = rf.fit(x_train, y_train)
model.score(x_test, y_test)

0.847

In [54]:
cv = cross_val_score(model, x, y, cv=4)
cv

array([0.84992, 0.78768, 0.298  , 0.7216 ])

Şimdi karmaşıklık matrisini oluşturup değerlere bakalım

In [71]:
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.98      0.92      8493
           1       0.47      0.10      0.17      1507

    accuracy                           0.85     10000
   macro avg       0.66      0.54      0.54     10000
weighted avg       0.80      0.85      0.80     10000



Şimdi ise farklı bir model ile sonuclara bakacağım 

In [56]:
xgbp = xgb.XGBClassifier()
xgb_model = xgbp.fit(x_train, y_train)
xgb_model.score(x_test, y_test)

0.8494

In [57]:
cv_xgb = cross_val_score(xgb_model, x_train, y_train, cv=4)
cv_xgb

array([0.8457, 0.8487, 0.8457, 0.8449])

In [70]:
y_pred_xgb = xgb_model.predict(x_test)
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
accuracy = accuracy_score(y_test, y_pred_xgb)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.86      0.98      0.92      8493
           1       0.50      0.11      0.17      1507

    accuracy                           0.85     10000
   macro avg       0.68      0.54      0.55     10000
weighted avg       0.81      0.85      0.81     10000



In [103]:
xgb_params = {
    'max_depth': [3, 4, 5, 6],  
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 500],
    'min_child_weight': [1, 5, 10],
    'gamma': [0, 0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 1.5, 2, 5]
}


In [104]:
xgb_model = xgb.XGBClassifier()

xgb_randomcv_model = RandomizedSearchCV(estimator=xgb_model, 
                                       param_distributions=xgb_params, 
                                       n_iter=100, 
                                       cv=5, 
                                       scoring='accuracy', 
                                       n_jobs=-1, 
                                       verbose=2).fit(x_train, y_train)


print('En iyi parametreler:', xgb_randomcv_model.best_params_)
print('En iyi accuracy score:', xgb_randomcv_model.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
En iyi parametreler: {'subsample': 1.0, 'reg_lambda': 5, 'reg_alpha': 0.5, 'n_estimators': 100, 'min_child_weight': 10, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 1, 'colsample_bytree': 0.6}
En iyi accuracy score: 0.851675


In [73]:
rc = RidgeClassifier()
rc_model = rc.fit(x_train, y_train)
rc_model.score(x_test,y_test)


0.8506

In [72]:
y_pred_rc = rc_model.predict(x_test)
cm_rc = confusion_matrix(y_test, y_pred_rc)
accuracy = accuracy_score(y_test, y_pred_rc)

print(classification_report(y_test, y_pred_rc))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92      8493
           1       0.59      0.03      0.06      1507

    accuracy                           0.85     10000
   macro avg       0.72      0.51      0.49     10000
weighted avg       0.81      0.85      0.79     10000



In [61]:
gb = GradientBoostingClassifier()
gb_model = gb.fit(x_train, y_train)
gb_model.score(x_test,y_test)

0.8495

In [62]:
cv_gb = cross_val_score(gb_model, x_train, y_train, cv=4)
cv_gb

array([0.8491, 0.8506, 0.8517, 0.8515])

In [63]:
lg = LogisticRegression()
lg_model = lg.fit(x_train,y_train)
lg_model.score(x_test,y_test)

0.8487

In [65]:
gb_params = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.6, 0.8, 1.0],
    'max_features': ['auto', 'sqrt', 'log2', None]
}
gb_model = GradientBoostingClassifier()


gb_randomcv_model = RandomizedSearchCV(estimator=gb_model, 
                                       param_distributions=gb_params, 
                                       n_iter=100, 
                                       cv=5, 
                                       scoring='accuracy', 
                                       n_jobs=-1, 
                                       verbose=2)


gb_randomcv_model.fit(x_train, y_train)


print('En iyi parametreler:', gb_randomcv_model.best_params_)
print('En iyi accuracy score:', gb_randomcv_model.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
En iyi parametreler: {'subsample': 0.6, 'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 6, 'learning_rate': 0.05}
En iyi accuracy score: 0.8515749999999999


In [64]:
lg_params = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': np.logspace(-4, 4, 20),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 300, 400, 500]
}


lg_model = LogisticRegression()
lg_randomcv_model = RandomizedSearchCV(estimator=lg_model,
                                       param_distributions=lg_params,
                                       n_iter=100,
                                       cv=5,
                                       scoring='accuracy',
                                       n_jobs=-1,
                                       verbose=2,
                                       random_state=42) 

lg_randomcv_model.fit(x_train, y_train)
print('En iyi parametreler:', lg_randomcv_model.best_params_)
print('En iyi accuracy score:', lg_randomcv_model.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
En iyi parametreler: {'solver': 'saga', 'penalty': 'l2', 'max_iter': 500, 'C': 11.288378916846883}
En iyi accuracy score: 0.8509500000000001
